## Эксперимент по вливанию коллекции в тематическую модель

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
%matplotlib inline
import os
import sys
import glob
import json
import shutil
import urllib
import pickle
import pymongo
import itertools
import collections
import numpy as np
import pandas as pd
import scipy.sparse
import matplotlib.pyplot as plt

In [6]:
import artm
import hierarchy_utils

In [7]:
db = pymongo.MongoClient()["datasets"]

In [8]:
prefix_to_col_map = {"pn": "postnauka", "habr": "habrahabr", "elem": "elementy"}

def get_document(doc_id, with_markdown=False):
    fields = {"_id": 1, "title": 1, "modalities": 1}
    if with_markdown:
        fields["markdown"] = 1
    prefix, _ = doc_id.split("_", 1)
    col_name = prefix_to_col_map[prefix]
    return db[col_name].find_one({"_id": doc_id}, fields)

In [9]:
# Шаг 1 -- получение ранжированного списка документов для доливания

reduced_dataset = False
shuffled_dataset = False
source_size = []

docs_ids = list(map(lambda r: r["_id"], db["postnauka"].find({}, {"_id": 1})))

D_a = len(docs_ids)
print(D_a)
source_size.append(D_a)

for csv_name, thresh in (("classifier_output_habr.csv", 0.95),
                         ("classifier_output_elem.csv", 0.8)):
    clf_output = pd.read_csv(csv_name)
    clf_output.columns = ["id", "proba"]
    clf_output = clf_output.set_index("id")["proba"]
    clf_output = clf_output[clf_output >= thresh]

    clf_output = clf_output.sort_values(ascending=False)

    if reduced_dataset:
        clf_output = clf_output[:5000]

    if shuffled_dataset:
        np.random.seed(42)
        clf_output = clf_output.sample(frac=1)

    docs_ids += list(clf_output.index)

    D_b = len(clf_output)
    source_size.append(D_b)
    print(D_b)

2976
1603
609


**В `docs_ids` лежит ранжированный список документов, который потом разбивается на батчи (`batches`)**

In [10]:
docs_ids[:10]

['pn_2',
 'pn_3',
 'pn_4',
 'pn_5',
 'pn_6',
 'pn_7',
 'pn_8',
 'pn_9',
 'pn_13',
 'pn_14']

**Дальше разбиваем на батчи. Можно по закону сложного процента:**

In [11]:
# Шаг 2а -- разбиение списка на батчи для итеративного доливания

D_a0 = D_a // 10

# Будем пользоваться законом сложного процента: размер i-ого батча будет считаться как размер (i-1)-ого батча + p%.
# Тогда количество итераций n, необходимых для вливания коллекции размера D_b в коллекцию размера D_a, удовлетворяет:
# D_a * (1 + p)^n = D_a + D_b
# n = ln((D_a + D_b) / D_a) / ln(1 + p)

p = 0.1
batches = [docs_ids[:D_a], docs_ids[D_a:D_a + D_a0]]
batch_pos = D_a + D_a0

while batch_pos < len(docs_ids):
    new_batch_size = int((1 + p) * len(batches[-1]))
    batch = docs_ids[batch_pos:batch_pos + new_batch_size]
    batches.append(batch)
    batch_pos += new_batch_size

**А можно просто слить всё в один батч:**

In [12]:
# Шаг 2б -- склеивание всей коллекции в один батч

batches = [docs_ids]

**А можно порезать все батчи, начиная со второго, на куски по 1000 штук:**

In [13]:
batches = [docs_ids[:D_a], docs_ids[D_a:D_a + 1000], docs_ids[D_a + 1000:D_a + 2000],
           docs_ids[D_a + 2000:D_a + 3000], docs_ids[D_a + 3000:D_a + 4000], docs_ids[D_a + 4000:D_a + 5000]]

**А можно сначала Постнауку, а потом -- всё остальное:**

In [14]:
batches = [docs_ids[:D_a], docs_ids[D_a:]]

In [15]:
batches = [docs_ids[:source_size[0]], docs_ids[source_size[0]:sum(source_size[:2])], docs_ids[sum(source_size[:2]):sum(source_size[:3])]]

**А может быть, можно ещё как-то -- тут надо думать.**

Размеры батчей:

In [16]:
list(map(len, batches))

[2976, 1603, 609]

Количество и суммарная длина батчей:

In [17]:
len(batches), sum(map(len, batches))

(3, 5188)

In [140]:
%%time

# Шаг 3 -- подгрузка словаря совстречаемостей (PPMI), подсчитываемого отдельного

artm_cooc_path = "merged_cooc_df_gt20.txt"

word_count = []
row, col, data = [], [], []
with open(artm_cooc_path) as cooc_f:
    for line in cooc_f:
        tokens = list(map(int, line.split()))
        if len(tokens) == 1:
            word_count.append(tokens[0])
        elif len(tokens) == 3:
            u, v, n_uv = tokens
            p_uv = n_uv / word_count[v]
            p_vu = n_uv / word_count[u]
            row.append(u); col.append(v); data.append(p_uv)
            row.append(v); col.append(u); data.append(p_vu)

w = h = len(word_count)
cooc_matrix = scipy.sparse.csr_matrix((data, (row, col)), shape=(w, h), dtype=np.float32)

del row, col, data

CPU times: user 1min 25s, sys: 21.5 s, total: 1min 47s
Wall time: 1min 40s


In [141]:
cooc_matrix

<231640x231640 sparse matrix of type '<class 'numpy.float32'>'
	with 38931218 stored elements in Compressed Sparse Row format>

In [21]:
# Параметры ARTM модели

# Общий random seed
artm_seed = 37

# Названия тем
# TODO: скрытые константы
norm_topic_names = ["topic_%d" % i for i in range(0, 20)]
background_names = ["background_%d" % i for i in range(0, 1)]
topic_names0 = norm_topic_names + background_names

# Регуляризаторы
regularizers_list0 = []
regularizers_list0.append(artm.DecorrelatorPhiRegularizer(name="DecorrPhiReg",
                                                          topic_names=norm_topic_names,
                                                          tau=100000))
regularizers_list0.append(artm.SmoothSparseThetaRegularizer(name="SPPhiTagRegBackground",
                                                           topic_names=background_names,
                                                           tau=10))

# Веса всех модальностей
class_ids0 = {"text": 1.0, "flat_tag": 300.0, "text_habr": 1.0, "flat_tag_habr": 1.0}
class_ids1 = {"text": 1.0, "flat_tag": 1.0,   "text_habr": 1.0, "flat_tag_habr": 1.0}

In [22]:
# Параметры алгоритма CollectionMerge

modalities_to_use = ["flat_tag", "text"]
vw_path = "batch_vw.txt"
artm_ppmi_path = "merged_ppmi.txt"
artm_vocab_path = "merged_vocab.txt"
artm_batch_path = "merged_batches/"
tmp_artm_batch_path = "./"

In [23]:
def build_level1(topics_cnt, level0, prev_phi, dictionary, batch_vectorizer, scores_list):
    # Названия тем
    background_names = ["background_%d" % i for i in range(0, 1)]
    norm_topic_names = ["topic_%d" % i for i in range(topics_cnt)]
    topic_names1 = norm_topic_names + background_names

    # Список регуляризаторов
    regularizers_list1 = []
    regularizers_list1.append(artm.DecorrelatorPhiRegularizer(name="DecorrPhiReg1",
                              topic_names=norm_topic_names, tau=50000))
    regularizers_list1.append(artm.SmoothSparseThetaRegularizer(name="SPPhiTagRegBackground",
                                                           topic_names=background_names,
                                                           tau=10))

    # Создадим ARTM модель (второй уровень)
    level1 = hierarchy_utils.ARTM_Level(level0, phi_batch_weight=10.0**3, 
                                        topic_names=topic_names1,
                                        class_ids=class_ids1,
                                        regularizers=regularizers_list1,
                                        scores=scores_list,
                                        num_document_passes=1,
                                        cache_theta=True, theta_columns_naming="title",
                                        seed=artm_seed)
    level1.initialize(dictionary=dictionary)

    # Инициализиуем матрицу Фи значениями с предыдущей итерации
    if prev_phi is not None:
        _, phi_ref = level1.master.attach_model(level1.model_pwt)
        print(prev_phi.shape, phi_ref.shape)
        for i, j in itertools.product(*map(range, prev_phi.shape)):
            phi_ref[i, j] = prev_phi[i, j]

    # Обучим модель второго уровня
    # TODO: подобрать количество итераций
    level1.fit_offline(batch_vectorizer, num_collection_passes=5)

    # Итеративное разреживание детей тем 0 уровня
    threshold = 0.05
    psi1 = level1.get_psi()
    for tau in np.arange(0.1, 0.6, 0.1):
        child_topics = []
        for t in range(len(level0.topic_names)):
            child_topics.append([])
            for s, topic_name1 in enumerate(level1.topic_names):
                if psi1.values[s, t] > threshold:
                    child_topics[t].append(topic_name1)
        # Разреживание детей каждой темы 0 уровня между собой
        for i in range(len(level0.topic_names)):
            topics_list = child_topics[i]
            level1.regularizers.add(artm.SmoothSparseThetaRegularizer(
                                    name="SPThetaReg_%d" % i,
                                    topic_names=topics_list, 
                                    tau=-tau * len(topics_list) ** 3), overwrite=True)
        # Дообучим модель второго уровня
        # TODO: подобрать количество итераций
        level1.fit_offline(batch_vectorizer, num_collection_passes=1)

    return level1

**Собственно, алгоритм итеративного обучения CollectionMerge:**

In [24]:
%%time

max_batch_size = max(map(len, batches))

vocab_map = {}
prev_phi0 = None
prev_phi1 = None

docs_length = 0
word_count = {}
pair_count = {}

all_chars = list(map(chr, range(ord('a'), ord('z') + 1)))
artm_batch_names_iter = itertools.product(*([all_chars] * 6))
batch_names = ["".join(next(artm_batch_names_iter)) for i in range(len(batches))]

t2 = 60

psis = []
levels = []
hiers = []

# Удалим содержимое директории с ARTM батчами
for fname in glob.glob(artm_batch_path + "/*"):
    os.remove(fname)

# Создадим файлы с ARTM vocabulary
open(artm_vocab_path, "w").close()

for iter_no, (batch_name, batch) in enumerate(zip(batch_names, batches)):
    prev_vocab_size = len(vocab_map)
    batch_vocab = []
    batch_n_d = collections.defaultdict(int)
    batch_n_wd = collections.defaultdict(int)

    # Запишем документы и словарь из батча в файлы
    with open(vw_path, "w") as vw_f, open(artm_vocab_path, "a") as vocab_f:
        for i_d, doc in enumerate(map(get_document, batch)):
            doc_id = doc["_id"]
            modalities_str = []
            for mod_name in modalities_to_use:
                mod = doc["modalities"].get(mod_name, [])
                mods = []
                for token in mod:
                    token = token.replace(" ", "_")
                    vocab_entry = (token, mod_name)
                    if iter_no == 0 or (iter_no > 0 and vocab_entry in vocab_map):
                        mods.append(token)
                    if iter_no == 0 and vocab_entry not in vocab_map:
                        vocab_map[vocab_entry] = len(vocab_map)
                        batch_vocab.append(vocab_entry)
                    #i_w = vocab_map[vocab_entry]
                    #batch_n_wd[i_w, i_d] += 1
                modalities_str.append("|%s %s" % (mod_name, " ".join(mods)))
                #for token in set(mod):
                #    token = token.replace(" ", "_")
                #    i_w = vocab_map[token, mod_name]
                #    batch_n_d[i_w] += 1
            vw_f.write("%s %s\n" % (doc_id, " ".join(modalities_str)))
        for vocab_entry in batch_vocab:
            vocab_f.write("%s %s\n" % vocab_entry)
    
    print("New vocab size:", len(batch_vocab))
    
    # Создадим матрицу P(w_old|d_new)
    """
    row, col, data = [], [], []
    for (i_w, i_d), n_wd in batch_n_wd.items():
        if i_w < prev_vocab_size:
            p_wd = n_wd / batch_n_d[i_w]
            row.append(i_w)
            col.append(i_d)
            data.append(p_wd)
    w = prev_vocab_size
    h = len(batch)
    batch_p_wd_old = scipy.sparse.csr_matrix((data, (row, col)), shape=(w, h), dtype=np.float32)
    
    # Создадим матрицу P(w_new|d_new)
    row, col, data = [], [], []
    for (i_w, i_d), n_wd in batch_n_wd.items():
        if i_w >= prev_vocab_size:
            p_wd = n_wd / batch_n_d[i_w]
            row.append(i_w - prev_vocab_size)
            col.append(i_d)
            data.append(p_wd)
    w = len(batch_vocab)
    h = len(batch)
    batch_p_wd_new = scipy.sparse.csr_matrix((data, (row, col)), shape=(w, h), dtype=np.float32)
    """

    # Создадим ARTM батч по нашему батчу
    tmp_batch_vectorizer = artm.BatchVectorizer(data_format="vowpal_wabbit", data_path=vw_path,
                                                batch_size=max_batch_size, target_folder=tmp_artm_batch_path,
                                                gather_dictionary=False)
    # Хак: переименуем ARTM батч во временное имя,
    # чтобы не мешать созданию других ARTM батчей
    os.rename("%s/aaaaaa.batch" % tmp_artm_batch_path,
              "%s/%s.batch"     % (artm_batch_path, batch_name))

    # Создадим ARTM модель (первый уровень)
    dictionary = artm.Dictionary("dictionary")
    dictionary.gather(artm_batch_path, vocab_file_path=artm_vocab_path,
                      cooc_file_path=artm_ppmi_path, symmetric_cooc_values=True)

    # Метрики качества
    scores_list = []
    scores_list.append(artm.PerplexityScore(name="PerplexityScore", class_ids=["text"]))
    scores_list.append(artm.TopTokensScore(name="Top50Tokens", class_id="text", num_tokens=50,
                                           dictionary=dictionary))
    scores_list.append(artm.TopTokensScore(name="Top10Tags", class_id="flat_tag", num_tokens=10,
                                           dictionary=dictionary))

    level0 = artm.ARTM(topic_names=topic_names0, class_ids=class_ids0,
                       regularizers=regularizers_list0, scores=scores_list,
                       cache_theta=True, theta_columns_naming="title",
                       seed=artm_seed)
    level0.initialize(dictionary=dictionary)
    
    # Загрузим созданные на данный момент ARTM батчи
    #data_paths = list(map(lambda i: "%s/%d" % (artm_batch_path, i), range(iter_no + 1)))
    #data_weights = list((1 - p) ** np.arange(iter_no + 1))
    batch_vectorizer = artm.BatchVectorizer(data_format="batches", data_path=artm_batch_path,
                                            gather_dictionary=False)
    
    _, phi_ref = level0.master.attach_model(level0.model_pwt)
    # Инициализиуем матрицу Фи значениями с предыдущей итерации
    if prev_phi0 is not None:
        # Инициализируем верхнюю часть Фи матрицей Фи с предыдущей итерации
        print(prev_phi0.shape, phi_ref.shape)
        for i, j in itertools.product(*map(range, prev_phi0.shape)):
            phi_ref[i, j] = prev_phi0[i, j]

        # Инициализируем нижнюю часть Фи различными оценками
        ff_score = 0.0
        if False:
            start_token_id = prev_phi0.shape[0]
            end_token_id = phi_ref.shape[0]

            for token_id in range(start_token_id, end_token_id):
                token_cooc_row = cooc_matrix[token_id]
                data, col = token_cooc_row.data, token_cooc_row.nonzero()[1]

                if len(data) > 0:
                    length = np.argmin(col < start_token_id)

                    pvt = prev_phi0[col[:length]]
                    pwv = data[:length]
                    pwt = pwv.dot(pvt)

                    # Посчитаем метрику качества инициализции
                    pvd_new = batch_p_wd_new[token_id - start_token_id]
                    pvd_old = batch_p_wd_old[col[:length]]
                    cf_score = pvd_new.toarray() - pvd_old.T.dot(pwv)
                    ff_score += np.linalg.norm(cf_score)

                    for j in range(prev_phi0.shape[1]):
                        phi_ref[token_id, j] = pwt[j]
                else:
                    pvd_new = batch_p_wd_new[token_id - start_token_id]
                    cf_score = pvd_new.toarray()
                    ff_score += np.linalg.norm(cf_score)
        #print("Level 0 FF score: %.6f" % ff_score)
    else:
        with open("phi_level0.dump", "rb") as fin:
            phi_level0 = pickle.load(fin)
            phi_ref[:] = phi_level0

    # Обучим модель первого уровня
    # TODO: подобрать количество итераций
    level0.fit_offline(batch_vectorizer, num_collection_passes=10)
    print("Level 0 perplexity: %.6f" % level0.score_tracker["PerplexityScore"].last_value)
    #print("Level 0 coherence: %.6f" % level0.score_tracker["Top50Tokens"].last_average_coherence)
    print("Iteration %d, level 0 built" % iter_no)

    # Построим модель второго уровня
    #max_coherence = 0
    #max_t2 = cur_t2
    #for t2 in range(cur_t2 - 4, cur_t2 + 12, 2):
    level1 = build_level1(t2, level0, prev_phi1, dictionary, batch_vectorizer, scores_list)
    print("Level 1 perplexity: %.6f" % level1.score_tracker["PerplexityScore"].last_value)
    #print("Level 1 coherence: %.6f" % level1.score_tracker["Top50Tokens"].last_average_coherence)
    print("Iteration %d, level 1 built" % iter_no)
    #if t2 == 19 * 3:
    #    t2 += 10

    # Сохраним матрицы Фи первого и второго уровня с текущей итерации
    prev_phi0 = level0.get_phi().values
    prev_phi1 = level1.get_phi().values

    # Сохраним уровни для истории
    #levels.append((level0, level1))
    
    # Сохраним иерархию
    hier = hierarchy_utils.hARTM(class_ids=class_ids0, regularizers=regularizers_list0,
                                 scores=scores_list, cache_theta=True, seed=artm_seed)

    hier._levels.append(level0)
    hier._levels.append(level1)
    hiers.append(hier)

New vocab size: 44995
Level 0 perplexity: 867.523369
Iteration 0, level 0 built
Level 1 perplexity: 546.795337
Iteration 0, level 1 built
New vocab size: 0
(44995, 21) (44995, 21)
Level 0 perplexity: 961.666048
Iteration 1, level 0 built
(44995, 61) (44995, 61)
Level 1 perplexity: 1611.299861
Iteration 1, level 1 built
New vocab size: 0
(44995, 21) (44995, 21)
Level 0 perplexity: 1084.252316
Iteration 2, level 0 built
(44995, 61) (44995, 61)
Level 1 perplexity: 2114.074653
Iteration 2, level 1 built
CPU times: user 14min 59s, sys: 30.1 s, total: 15min 29s
Wall time: 12min 33s


In [26]:
hiers[-1].save("hartm2")

extra_info = {
    "class_ids": class_ids0,
    "theta": hiers[-1].get_theta(),
    "spectrums": []
}

pickle.dump(extra_info, open("hartm2/extra_info.dump", "wb"))

**Получившиеся темы первого уровня:**

In [33]:
sorted([(k, " ".join(v[:3])) for k, v in hiers[-1]._levels[0].score_tracker["Top10Tags"].last_tokens.items()])

[('background_0', 'общество антропология человек'),
 ('topic_0', 'математика логика кибернетика'),
 ('topic_1', 'технологии искусственный_интеллект информационные_технологии'),
 ('topic_10', 'история история_россии европа'),
 ('topic_11', 'политика государство политология'),
 ('topic_12', 'социология город урбанистика'),
 ('topic_13', 'культура массовая_культура культурология'),
 ('topic_14', 'образование наука история_науки'),
 ('topic_15', 'язык лингвистика филология'),
 ('topic_16', 'философия политическая_философия древняя_греция'),
 ('topic_17', 'христианство средневековье религия'),
 ('topic_18', 'россия украина география'),
 ('topic_19', 'право юриспруденция римское_право'),
 ('topic_2', 'физика физика_элементарных_частиц квантовая_физика'),
 ('topic_3', 'химия нанотехнологии материаловедение'),
 ('topic_4', 'земля экология солнечная_система'),
 ('topic_5', 'астрономия астрофизика вселенная'),
 ('topic_6', 'биология генетика эволюция'),
 ('topic_7', 'медицина онкология биомедици

In [36]:
sorted([(k, " ".join(v[:10])) for k, v in hiers[-1]._levels[0].score_tracker["Top50Tokens"].last_tokens.items()])

[]

**Получившиеся темы второго уровня:**

In [37]:
sorted([(k, " ".join(v[:3])) for k, v in hiers[-1]._levels[1].score_tracker["Top10Tags"].last_tokens.items()])

[('background_0', 'технологии искусственный_интеллект биотехнологии'),
 ('topic_0', 'онкология старение генетическое_заболевание'),
 ('topic_1', 'биология ген молекулярная_биология'),
 ('topic_10', 'онкология стволовые_клетки вирусология'),
 ('topic_11', 'физика атом нейтрино'),
 ('topic_12', 'ньютон_исаак суперкомпьютеры научная_фантастика'),
 ('topic_13', 'астрономия звезды вселенная'),
 ('topic_14', 'нейробиология когнитивная_психология нейрофизиология'),
 ('topic_15', 'физика физика_элементарных_частиц бозон_хиггса'),
 ('topic_16', 'востоковедение католицизм кавказ'),
 ('topic_17', 'массовая_культура культурология театроведение'),
 ('topic_18', 'биология геном ген'),
 ('topic_19', 'лингвистика язык филология'),
 ('topic_2', 'физика элементарная_частица бозон_хиггса'),
 ('topic_20', 'нейробиология нейронные_сети эмоции'),
 ('topic_21', 'фольклористика культурология фольклор'),
 ('topic_22', 'биология ген геном'),
 ('topic_23', 'авторское_право теория_принятия_решений финансовый_криз

In [84]:
#for level_id, level in enumerate(levels):
#    level.save("article_models/new_random_small_batches_%d.model" % level_id)

---

---

**Какие-то статистики (в том числе и из старой Машиной тетрадки), которые можно позапускать без гарантии успеха:**

### Proposed algorithm

### Baseline

In [ ]:
phi0 = levels[0].get_phi()

In [ ]:
np.mean((phi0 > 0).sum(axis=1))

In [ ]:
np.median((phi0 > 0).sum(axis=1))

In [ ]:
sorted([(k, " ".join(v[:3])) for k, v in levels[0].score_tracker["Top50Tokens"].last_tokens.items()])

In [ ]:
sorted([(k, " ".join(v[:3])) for k, v in levels[-1].score_tracker["Top10Tags"].last_tokens.items()])

---

In [150]:
level0, level1 = levels[-1]
psi1 = level1.get_psi()

tokens0 = level0.score_tracker["Top10Tags"].last_tokens
tokens1 = level1.score_tracker["Top10Tags"].last_tokens
threshold = 0.03

In [151]:
child_topics = []
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    child_topics.append([])
    output += topic_name + ': '
    for word in tokens0[topic_name][:10]:    
        output += word + ' '
    print("\x1b[1;31m PARENT: ", output, '\x1b[0m')
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if (psi1.values[s, t] > threshold):
            child_topics[t].append(topic_name1)
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1][:10]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

related_topics = 0
for row in (psi1.values>threshold):
    if (sum(row) != 0):
        related_topics += 1

print(related_topics,"topics have parents")    

 PARENT:  topic_0: философия политическая_философия история_философии русь античность археология платон аристотель история_россии архитектура  
    topic_0: политическая_философия история_философии петр_i древняя_греция древний_рим русская_философия политическая_теория екатерина_ii кавказ гоббс_томас 
    topic_21: история_россии кавказ история политический_режим академия_наук критика_насилия геласий_i гуманитарная_культура бонифаций_viii протестантизм 
    topic_27: наука история_науки гендер академическая_среда кант_иммануил философия_науки социология_права либерализм российская_академия_наук фуко_мишель 
 PARENT:  topic_1: эволюция антропология биология человек палеонтология геология антропогенез физиология происхождение_человека этнография  
    topic_14: антропология человек антропогенез происхождение_человека расоведение теория_эволюции неандерталец homo_sapiens дарвин_чарльз метеорология 
    topic_45: биология человек физическая_антропология антропогенез антропология дарвин_чар

In [107]:
child_topics = []
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    child_topics.append([])
    output += topic_name + ': '
    for word in tokens0[topic_name][:10]:    
        output += word + ' '
    print("\x1b[1;31m PARENT: ", output, '\x1b[0m')
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if (psi1.values[s, t] > threshold):
            child_topics[t].append(topic_name1)
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1][:10]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

related_topics = 0
for row in (psi1.values>threshold):
    if (sum(row) != 0):
        related_topics += 1

print(related_topics,"topics have parents")    

 PARENT:  topic_0: история_россии философия история политическая_философия история_философии античность археология русь платон аристотель  
    topic_0: философия античность археология русь аристотель платон архитектура украина россия этика 
    topic_21: философия религия смерть русский политический имя эпоха древний бог свобода 
    topic_27: философия университет марксизм феминизм идеология водород закон ученый капитализм дворянство 
 PARENT:  topic_1: эволюция антропология биология человек палеонтология геология антропогенез физиология происхождение_человека расоведение  
    topic_14: эволюция палеонтология физиология геология иммунология этнография неврология этнос раса homo 
    topic_20: вид группа развитие общий часть давать связь возникать показывать результат 
    topic_45: геология эволюция сон признак изменение древний жить период группа форма 
 PARENT:  topic_2: экономика география информационная_безопасность авторское_право финансовый_кризис финансовый_рынок экономическо

### Модель Постнауки

In [116]:
level0.score_tracker["PerplexityScore"].value[-5:]

[4014.377057868622,
 3997.412386903887,
 3983.4014716661077,
 3971.371280361148,
 3961.1586255593693]

In [117]:
level1.score_tracker["PerplexityScore"].value[-5:]

[1658.0593815180423,
 1690.214988651034,
 1735.0400707548297,
 1774.8115995426745,
 1808.6524507405409]

In [118]:
print("\n".join([k + ": " + " ".join(v) for k, v in level0.score_tracker["Top10Tokens"].last_tokens.items()]))

KeyError: 'Top10Tokens'

In [119]:
print("\n".join([k + ": " + " ".join(v) for k, v in level1.score_tracker["Top10Tokens"].last_tokens.items()]))

KeyError: 'Top10Tokens'

In [ ]:
level0.score_tracker["Top10Tokens"].last_average_coherence

In [ ]:
level1.score_tracker["Top10Tokens"].last_average_coherence

### Предлагаемый алгоритм

In [ ]:
level0.score_tracker["PerplexityScore"].value[-5:]

In [ ]:
level1.score_tracker["PerplexityScore"].value[-5:]

In [ ]:
print("\n".join([k + ": " + " ".join(v) for k, v in level0.score_tracker["Top10Tokens"].last_tokens.items()]))

In [ ]:
print("\n".join([k + ": " + " ".join(v) for k, v in level1.score_tracker["Top10Tokens"].last_tokens.items()]))

In [ ]:
level0.score_tracker["Top10Tokens"].last_average_coherence

In [ ]:
level1.score_tracker["Top10Tokens"].last_average_coherence

In [ ]:
%%time

docs_lengths = {}

for doc in map(get_document, docs_ids):
    docs_lengths[doc["_id"]] = len(doc["modalities"]["text"])

In [ ]:
theta0 = level0.get_theta()

docs_lengths_series = pd.Series(docs_lengths)
pn_docs_ids = list(filter(lambda doc_id: doc_id.startswith("pn_"), docs_ids))
n_pn_topics = theta0[pn_docs_ids].dot(docs_lengths_series.loc[pn_docs_ids])
n_topics = theta0.dot(docs_lengths_series)
n_topics /= n_topics.sum()

plt.figure(figsize=(12, 8))
plt.scatter(n_topics[:-1], n_pn_topics[:-1])
plt.xlabel("$p(t)$")
plt.ylabel("$n_t^{ПН}$")
#plt.xlim([0, 0.02])
#plt.ylim([0, 25000])
plt.show()

In [ ]:
theta1 = level1.get_theta()

docs_lengths_series = pd.Series(docs_lengths)
pn_docs_ids = list(filter(lambda doc_id: doc_id.startswith("pn_"), docs_ids))
n_pn_topics = theta1[pn_docs_ids].dot(docs_lengths_series.loc[pn_docs_ids])
n_topics = theta1.dot(docs_lengths_series)
n_topics /= n_topics.sum()

plt.figure(figsize=(12, 8))
plt.scatter(n_topics, n_pn_topics)
plt.xlabel("$p(t)$")
plt.ylabel("$n_t^{ПН}$")
#plt.xlim([0, 0.02])
#plt.ylim([0, 25000])
plt.show()

### Базовый алгоритм

In [ ]:
level0.score_tracker["PerplexityScore"].value[-5:]

In [ ]:
level1.score_tracker["PerplexityScore"].value[-5:]

In [ ]:
print("\n".join([k + ": " + " ".join(v) for k, v in level0.score_tracker["Top10Tokens"].last_tokens.items()]))

In [ ]:
print("\n".join([k + ": " + " ".join(v) for k, v in level1.score_tracker["Top10Tokens"].last_tokens.items()]))

In [ ]:
level0.score_tracker["Top10Tokens"].last_average_coherence

In [ ]:
level1.score_tracker["Top10Tokens"].last_average_coherence

In [ ]:
%%time

docs_lengths = {}

for doc in map(get_document, docs_ids):
    docs_lengths[doc["_id"]] = len(doc["modalities"]["text"])

In [ ]:
theta0 = level0.get_theta()

docs_lengths_series = pd.Series(docs_lengths)
pn_docs_ids = list(filter(lambda doc_id: doc_id.startswith("pn_"), docs_ids))
n_pn_topics = theta0[pn_docs_ids].dot(docs_lengths_series.loc[pn_docs_ids])
n_topics = theta0.dot(docs_lengths_series)
n_topics /= n_topics.sum()

plt.figure(figsize=(12, 8))
plt.scatter(n_topics[:-1], n_pn_topics[:-1])
plt.xlabel("$p(t)$")
plt.ylabel("$n_t^{ПН}$")
#plt.xlim([0, 0.02])
#plt.ylim([0, 25000])
plt.show()

In [ ]:
theta1 = level1.get_theta()

docs_lengths_series = pd.Series(docs_lengths)
pn_docs_ids = list(filter(lambda doc_id: doc_id.startswith("pn_"), docs_ids))
n_pn_topics = theta1[pn_docs_ids].dot(docs_lengths_series.loc[pn_docs_ids])
n_topics = theta1.dot(docs_lengths_series)
n_topics /= n_topics.sum()

plt.figure(figsize=(12, 8))
plt.scatter(n_topics, n_pn_topics)
plt.xlabel("$p(t)$")
plt.ylabel("$n_t^{ПН}$")
#plt.xlim([0, 0.02])
#plt.ylim([0, 25000])
plt.show()

**NB**: в конце провести эксперимент на перемешанном clf_output -- гипотеза о независимости качества от порядка вливания документов в ТМ.

---

---